## Obtain Data

In this notebook we perform the following steps:
* Establish the first hour of the dataset
* For the first month,
  * Obtain a list of available stations by state
  * Obtain temperature observations from weather stations in the MISO footprint
    * Stations are organized into MISO regions by state boundaries
    * Stations are predominantly clustered in population centers, making many observation redundant
    * There are many lacunae in some stations
  * Obtain the actual hourly MISO Load data and historical Medium-Term Load Forecasts (MTLF)
  * Join Load and MTLF data with weather observations to complete the raw data
  * Persist the data and demonstrate use of dataset wrapper class
* Update the dataset to the current day
* Identify and mitigate lacunae
* Publish the data

### Define Date Ranges

In [ ]:
from datetime import datetime, timezone, timedelta
def est(yyyy, mm, dd, hh):
    return datetime(yyyy, mm, dd, hh, tzinfo=timezone(timedelta(hours=-5)))

# beginning of MISO's historical records that include the southern region (zones 8-10)
first_hour = est(2015, 2, 1, 0)

# latest date with actual load data available is
# l = date.today() - timedelta(days=2)
# last_hour = est(l.year, l.month, l.day, 23)
# instead, fix the date for repeatbility
last_hour = est(2022, 4, 22, 23)

test_split = last_hour - timedelta(days=364, hours=23)
validation_split = test_split - timedelta(days=365)


### Obtain Weather Data for Zones

[Iowa State ASOS Network Downloads](https://mesonet.agron.iastate.edu/request/download.phtml)

In [ ]:
from weather_data import get_station_df
from pathlib import Path
from os.path import isfile
import pandas as pd

zones = { 1 : {'MN': ['MSP',  # Minneapolis / St. Paul (STP)
                      'RST',  # Rochester
                      'DLH'], # Duluth
               'ND': ['FAR',  # Fargo
                      'BIS',  # Bismarck
                      'GFK'], # Grand Forks
               'SD': ['ABR'], # Aberdeen
               'WI': ['LSE'], # La Crosse
               'IL': ['SFY']  # Savanna 
              },
          2 : {'WI': ['MSN', 'MKE', 'EAU', 'GRB'],
               'MI': ['ANJ', 'SAW', 'IWD']},
          3 : {'IA': ['DSM', 'CID', 'DVN', 'SUX', 'ALO', 'MCW']}
        }

def download_file_path(zone, state, station):
    zone_data = f"./data/zone_{zone}"
    Path(zone_data).mkdir(exist_ok=True)
    return f'{zone_data}/{state}_{station}.parquet'

def download_station(zone, state, station):
    path = download_file_path(zone, state, station) 
    if isfile(path):
        return pd.read_parquet(path)

    station = get_station_df(station, first_hour, last_hour)
    if station is None:
        print(f'Retrieve {station} failed')
        return None
    return station.to_parquet(path)

from multiprocessing import cpu_count
from joblib import Parallel, delayed
def do_parallel(func):
    parallel = Parallel(n_jobs=cpu_count())
    result = {}
    for zone in zones:
        stations = [(state, station) for state in zones[zone].keys() for station in zones[zone][state]]
        result[zone] = parallel(delayed(func)(zone, state, station) for (state, station) in stations)
    return result

In [ ]:
_ = do_parallel(download_station)

We have obtained raw weather observations at various intervals, usually 15 minutes, but there is a lot of missing data.

In [ ]:
raw_df = pd.DataFrame()
for zone in zones:
  for state in zones[zone]:
      for station in zones[zone][state]:
        raw_df = pd.concat([raw_df, pd.read_parquet(download_file_path(zone, state, station))])
raw_df[raw_df['tmpf'] == 'M'].head()

In [ ]:
raw_df[raw_df['tmpf'] == 'M'].shape

We need a temperature observation for each hour, since that is how the MISO MTLF is reported. Our initial approach will be to simply drop all missing observations, then choose the observation that is closest in time to the top of the hour.

In [ ]:
observation_dates = pd.date_range(start = first_hour, end = last_hour)
observation_hours = [d.replace(hour = h) for d in observation_dates for h in range(0, 24)]

def build_hourly_df(zone, state, station):
    w = pd.read_parquet(download_file_path(zone, state, station))
    df = w[w['tmpf'] != 'M'].copy()
    df['valid'] = pd.to_datetime(df['valid'], utc=True)
    numeric_cols = ['tmpf', 'lat', 'lon']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, axis=1)
    df = df.drop(columns=['feel'])
    idx = df.drop_duplicates('valid').set_index('valid').index.get_indexer(observation_hours, method='nearest')
    print(pd.Series(idx).value_counts())
    df = df.iloc[idx]
    df.loc[: , 'valid'] = df['valid'].dt.round(freq='H')
    return df.drop_duplicates('valid') 


In [ ]:
dfs = do_parallel(build_hourly_df)
zonal_weather_data = {}
for zone in zones:
    zonal_weather_data[zone] = pd.concat(dfs[zone], ignore_index=True)

In [ ]:
for station in pd.unique(zonal_weather_data[1]['station']):
    df = zonal_weather_data[1]
    count = df[df['station'] == station].shape[0]
    print(f'{station} observation count {count}')
print(f'Required observations {len(observation_hours)}')

In [ ]:
merged = pd.merge(df[df['station'] == 'DLH'], pd.DataFrame(observation_hours, columns=['Hours']), how='right', left_on='valid', right_on='Hours')
merged[merged['station'].isna()]['Hours'].apply(lambda h: h.date).value_counts()[0:100]

There are entire days missing in the observations. We need to try to obtain observations from nearby stations to fill in the lacunae.

In [ ]:
import pandas as pd
from weather_data import get_stations, get_session
duluth = pd.read_parquet(download_file_path(1, 'MN', 'DLH'))
stations_raw = get_stations('MN', 2015, get_session('https://'))

In [ ]:
stations_MN_df = pd.DataFrame(stations_raw)

In [ ]:
stations_MN_df.head()

In [16]:
# sort stations MN by closest to another station
from geopy import distance
dlh = stations_MN_df[stations_MN_df['sid'] == 'DLH'].iloc[0]['latlon']
stations_MN_df['distance_from_DLH'] = stations_MN_df['latlon'].apply(lambda c: distance.distance(dlh, c).mi)

In [17]:
stations_MN_df.sort_values(['distance_from_DLH'])

,elevation,sname,time_domain,state,country,climate_site,wfo,tzname,ncdc81,ncei91,ugc_county,ugc_zone,county,sid,latlon,distance_from_DLH
19,435.0,DULUTH,(1948-Now),MN,US,MN2248,DLH,America/Chicago,USW00014913,USW00014913,MNC137,MNZ037,St Louis,DLH,"(46.8421, -92.1936)",0.000000
20,186.0,DULUTH (SKY HARB,(1976-Now),MN,US,MN2248,DLH,America/Chicago,USC00478349,USC00478349,MNC137,MNZ037,St. Louis,DYT,"(46.7219, -92.0434)",10.942301
13,390.0,CLOQUET,(1991-Now),MN,US,MN1630,DLH,America/Chicago,USC00211630,USC00211630,MNC017,MNZ037,St. Louis,COQ,"(46.7011, -92.5036)",17.644290
91,328.0,TWO HARBORS,(1991-Now),MN,US,MN8419,DLH,America/Chicago,USC00218419,USC00218419,MNC075,MNZ020,Lake,TWM,"(47.0492, -91.745)",25.593478
59,328.0,MOOSE LAKE,(1976-Now),MN,US,MN1074,DLH,America/Chicago,USC00215598,USC00215598,MNC017,MNZ037,Carlton,MZH,"(46.4186, -92.8048)",41.249770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,441.0,JACKSON MUNI,(1991-Now),MN,US,MN9033,FSD,America/Chicago,USC00214453,USC00214453,MNC063,MNZ900,Jackson,MJQ,"(43.65, -94.9866)",259.105153
81,495.0,Slayton,(2005-Now),MN,US,MN8323,FSD,America/Chicago,USC00214534,USC00214534,MNC101,MNZ900,Murray,DVP,"(43.9868, -95.7826)",263.293540
101,480.0,WORTHINGTON,(1972-Now),MN,US,MN9033,FSD,America/Chicago,USC00219170,USC00219170,MNC105,MNZ900,Nobles,OTG,"(43.6551, -95.5792)",275.100258
70,529.0,PIPESTONE,(1991-Now),MN,US,MN6565,FSD,America/Chicago,USC00216565,USC00216565,MNC117,MNZ900,Pipestone,PQN,"(43.9821, -96.3004)",280.838537


In [ ]:
dlh_latlon

In [ ]:
distance.distance(dlh_latlon, dlh_latlon)

### Obtain the Regional MTLF and Actual Load for each Observation Hour

In [ ]:
from rf_al_data import 

Path("./data/mtlf").mkdir(exist_ok=True)
forecast_output_dir = './data/mtlf'
# the actuals aren't available until the next day
actuals = get_daily_rf_al_df(first_hour, last_hour + timedelta(days=2.0), forecast_output_dir)
actuals

### Harmonize Features with Actuals

There are a number of lacunae in the weather observations.

In [ ]:
def mktime_idx(row): 
    return datetime.combine(row['Market Day'].date(), time(row['HourEnding'] - 1), timezone(timedelta(hours = -5)))

actuals['time_idx'] = actuals.apply(mktime_idx, axis = 1)

In [ ]:
actuals.to_parquet('./data/actuals_mtlf.parquet')
actuals

In [ ]:
p = df.pivot(index='valid', columns='station', values='tmpf').dropna()
data = p.join(actuals.set_index('time_idx'), how='inner')

(n, _) = data.shape
(num_weather_observations, _) = df.groupby('valid').count().shape
(num_load_observations, _) = actuals.shape
(num_weather_observations, num_load_observations, len(observation_hours), n)

## Feature Engineering: Business Hours

Can we improve the performance of the model by introducing business hours into the feature set?

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay, BusinessHour

federal_business_days = CustomBusinessDay(calendar=USFederalHolidayCalendar())
bh = BusinessHour()
def is_biz_hour(d):
    return federal_business_days.is_on_offset(d) and bh.is_on_offset(d)
data['IsBusinessHour'] = data.index.to_series().apply(lambda d: 1 if is_biz_hour(d) else 0)